# Political Weather Map
This project collects daily news from around the world and displays them on a heat map by category, by using data that GDELT publishes on Google BigQuery for free.
- Member: Charlotte Bacchetta, Samuel Bennett, Hiroyuki Oiwa
- Source: [GDELT](https://www.gdeltproject.org/data.html#rawdatafiles)

## 1. Instal Google Cloud BigQuery libraries.


In [23]:
!pip install google-cloud-bigquery pandas

## 2. Load Data

In [24]:
from google.colab import auth
auth.authenticate_user()

In [25]:
from google.cloud import bigquery
PROJECT_ID = "week8-439513"
client = bigquery.Client(project=PROJECT_ID)

In [26]:
query = """
SELECT *
FROM `gdelt-bq.gdeltv2.ggg`
WHERE TIMESTAMP_TRUNC(DateTime, DAY) = TIMESTAMP("2025-02-04")
LIMIT 1000
"""
df = client.query(query).to_dataframe()

In [27]:
df.sample(5)

,DateTime,URL,Title,SharingImage,LangCode,DocTone,DomainCountryCode,Location,Lat,Lon,CountryCode,Adm1Code,Adm2Code,GeoType,ContextualText,the_geom
184,2025-02-04 22:45:00+00:00,https://saharareporters.com/2025/02/04/oped-pe...,# OpEd : Peter Obi Attachment To The Fraudule...,None,eng,-2.500000,NI,Niger,16.0000,8.000,NG,NG,None,1,in the diaspora on top of that he is an elder ...,POINT(8 16)
6,2025-02-04 02:30:00+00:00,http://www.israelnationalnews.com/news/403358,None,https://a7.org/files/pictures/1200x930/1176236...,eng,0.508475,IS,Canada,60.0000,-96.000,CA,CA,None,1,cartels as terrorist organizations and launchi...,POINT(-96 60)
870,2025-02-04 23:00:00+00:00,https://www.woodstocksentinelreview.com:443/ne...,' Creativity ' key to curbing memorial renewa...,None,eng,-0.645161,CA,"Sarnia, Ontario, Canada",42.9667,-82.400,CA,CA08,12661,4,creativity key to curbing memorial renewal cos...,POINT(-82.4 42.9667)
867,2025-02-04 22:45:00+00:00,https://www.theslateonline.com/article/2025/02...,' Cowboy Carter ' takes country crown at 67th...,https://snworksceo.imgix.net/slt/301807b2-2119...,eng,5.882353,US,"Los Angeles, California, United States",34.0522,-118.244,US,USCA,CA037,3,brad paisley st vincent brittany howard john l...,POINT(-118.244 34.0522)
876,2025-02-04 23:00:00+00:00,https://www.woodstocksentinelreview.com:443/ne...,' Creativity ' key to curbing memorial renewa...,None,eng,-0.645161,CA,"Sarnia, Ontario, Canada",42.9667,-82.400,CA,CA08,12661,4,to make things right but we dont want to be th...,POINT(-82.4 42.9667)


## 3. Extract Necessary Data

In [28]:
import pandas as pd

articles = pd.DataFrame()

articles['DateTime'] = df['DateTime']
articles['CountryCode'] = df['CountryCode']
articles['Title'] = df['Title']
articles['ContextualText'] = df['ContextualText']
articles['URL'] = df['URL']

articles.sample(5)

,DateTime,CountryCode,Title,ContextualText,URL
579,2025-02-04 14:30:00+00:00,VE,' Beyond betrayal .' Venezuelans in Florida a...,they could be targeted for deportation by fede...,https://www.krwg.org/national-news/2025-02-04/...
744,2025-02-04 14:30:00+00:00,VE,' Beyond betrayal .' Venezuelans in Florida a...,venezuelan migrants in south florida say they ...,https://www.krwg.org/national-news/2025-02-04/...
37,2025-02-04 04:15:00+00:00,IS,None,he has the potential to accomplish what previo...,https://www.israelnationalnews.com/news/403364
907,2025-02-04 05:30:00+00:00,US,' Day Without Immigrant movement takes place ...,jafary reporter modesto demonstrators join in ...,https://www.kcra.com/article/modesto-day-witho...
651,2025-02-04 14:30:00+00:00,VE,' Beyond betrayal .' Venezuelans in Florida a...,leaders to denounce changes to the protections...,https://www.kawc.org/npr-news/2025-02-04/beyon...


## 4. Visualization

In [29]:
articles_groupby = articles.groupby('CountryCode')['DateTime'].count().reset_index(name='Count')
articles_groupby.sample(5)

,CountryCode,Count
8,CU,3
50,US,172
30,NG,25
52,VM,2
39,QA,3


In [30]:
# The country coude mapping to convert country codes to ISO alpha2 style is generatede by ChatGPT.
country_code_mapping = {
    'EI': 'IE',
    'EZ': 'ES',
    'IZ': 'IT',
    'HA': 'HR',
    'LO': 'LI',
    'OS': 'AT',
}

articles_groupby['CountryCode'] = articles_groupby['CountryCode'].replace(country_code_mapping)

In [31]:
!pip install pycountry

In [32]:
import plotly.express as px
from pycountry import countries

# This convert function is generatede by ChatGPT.
def convert_to_alpha3(alpha2_code):
    try:
        return countries.get(alpha_2=alpha2_code).alpha_3
    except AttributeError:
        return None

articles_groupby['CountryCode'] = articles_groupby['CountryCode'].apply(convert_to_alpha3)

fig = px.choropleth(articles_groupby,
                    locations="CountryCode",
                    color="Count",
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title="Articles per Country on 2025-02-04",
                    width=700,
                    height=500
                    )

fig.update_layout(coloraxis_colorbar_title="Number of Articles")

fig.show()